<a href="https://colab.research.google.com/github/tractorjuice/Building_BoK/blob/main/Building_Wardley_Mapping_Body_of_Knowledge_Part_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wardley Mapping Body of Knowledge Using Langchain & OpenAI
## Part 4, query the vector database using ChatGPT

This example shows how to create and query an internal knowledge base using ChatGPT.

This does not require a GPU runtime.

## Set Up


Use Pinecone for the Vector Database

In [ ]:
from tqdm.autonotebook import tqdm
!pip install -q langchain
!pip install -q pinecone-client
!pip install -q openai
!pip install -q tiktoken
from langchain.vectorstores import Pinecone
from tqdm.autonotebook import tqdm
import pinecone

In [ ]:
# initialize pinecone
pinecone.init(
    api_key="",  # find at app.pinecone.io
    environment=""  # next to api key in console
    )

index_name = "" # Put your Pinecone index name here
name_space = "" # Put your Pinecone namespace here


### Set up OPEN_API_KEY and necessary variables

In [ ]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = "" # Put your OpenAI API key here

# Query using the vector store with ChatGPT integration

Setup access to the Pinecone vector database

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()
vector_store = Pinecone.from_existing_index(index_name, embeddings, namespace=name_space)

Setup the prompt

In [ ]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

system_template="""
    You are SimonGPT a strategy researcher based in the UK.
    “Researcher” means in the style of a strategy researcher with well over twenty years research in strategy and cloud computing.
    You use complicated examples from Wardley Mapping in your answers, focusing on lesser-known advice to better illustrate your arguments.
    Your language should be for an 12 year old to understand.
    If you do not know the answer to a question, do not make information up - instead, ask a follow-up question in order to gain more context.
    Use a mix of technical and colloquial uk englishlanguage to create an accessible and engaging tone.
    Provide your answers using Wardley Mapping in a form of a sarcastic tweet.
    Use the following pieces of context to answer the users question.
    Take note of the sources and include them in the answer in the format: "SOURCES: source1 source2", use "SOURCES" in capital letters regardless of the number of sources.
----------------
{summaries}
"""
messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template("{question}")
]
prompt = ChatPromptTemplate.from_messages(messages)

Initialise the LLM API

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQAWithSourcesChain

chain_type_kwargs = {"prompt": prompt}
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, max_tokens=256)  # Modify model_name if you have access to GPT-4
chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)


#### Use the chain to query

In [ ]:
query = "what is an ecosystem?"
result = chain(query)


Print the sources so we can find the YouTube videos

In [ ]:
print(result['question'])
print(result['answer'])
print(result['sources'])

source_documents = result['source_documents']
for index, document in enumerate(source_documents):
    print(f"Source {index + 1}:")
    print(f"  Page Content: {document.page_content}")
    print(f"  Source: {int(document.metadata['source'])}")
    print(f"  Source URL: https://www.youtube.com/watch?v={document.metadata['source_url']}&t={int(document.metadata['source'])}")

    print(f"  Source Title: {document.metadata['title']}")
    print(f"  Source Author: {document.metadata['author']}")
